In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_12_CLEANED.csv")
tsr_12 = pd.read_csv(csv_path)
tsr_12.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1,mrs_tx_3,mrs_tx_6,mrs_tx_12
0,153.0,62.0,3,1,3,0,0.0,8.000,0,999,...,0,1,0,4,0,67.0,1,1,1,1
1,152.0,62.0,3,1,2,0,0.0,4.000,0,999,...,0,0,0,1,0,69.0,1,0,0,0
2,148.0,56.0,2,1,2,0,0.0,5.000,0,999,...,0,0,0,2,0,71.0,0,0,0,0
3,152.0,56.0,4,1,2,0,0.0,3.000,1,0,...,0,0,0,0,0,71.0,0,0,0,0
4,170.0,87.8,5,8,3,0,0.0,8.491,0,999,...,0,0,2,2,1,59.0,5,5,5,4


In [3]:
tsr_12_input = tsr_12.drop(["mrs_tx_12"], axis=1)
tsr_12_input[tsr_12_input == "N"] = 0
tsr_12_input[tsr_12_input == "Y"] = 1
tsr_12_input = tsr_12_input.astype("float64")
tsr_12_input = np.array(tsr_12_input.values)

tsr_12_input_nomrs = tsr_12.drop(["mrs_tx_12", "mrs_tx_6", "mrs_tx_3", "mrs_tx_1"], axis=1)
tsr_12_input_nomrs[tsr_12_input_nomrs == "N"] = 0
tsr_12_input_nomrs[tsr_12_input_nomrs == "Y"] = 1
tsr_12_input_nomrs = tsr_12_input_nomrs.astype("float64")
tsr_12_input_nomrs = np.array(tsr_12_input_nomrs.values)

# 6 classes

In [4]:
tsr_12_output = tsr_12.mrs_tx_1
tsr_12_output = tsr_12_output.astype("float64")
tsr_12_output = np.array(tsr_12_output.values)

## SVM

In [5]:
svr = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores = cross_val_score(svr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores)
print("Mean of R^2:", svr_scores.mean())
print("Std of R^2:", svr_scores.std())

[0.97750597 0.95628404 0.97446699 0.92790661 0.964881   0.97956609
 0.98262851 0.91944339 0.93719758 0.96678132]
Mean of R^2: 0.9586661505309936
Std of R^2: 0.021613042221545108


In [6]:
svr.fit(tsr_12_input,tsr_12_output)
svr_predict =svr.predict(tsr_12_input)
print(svr_predict)
svr_predict = np.round(svr_predict)
print(svr_predict)
print("Accuracy of RF:", (tsr_12_output == svr_predict).sum() / len(svr_predict))

[1.07725649 0.73680547 0.4681583  ... 0.51904241 4.76491759 1.21413784]
[1. 1. 0. ... 1. 5. 1.]
Accuracy of RF: 0.7922774401144083


In [7]:
svr_pred = cross_val_predict(svr,tsr_12_input,tsr_12_output,cv = 10)
svr_pred = np.round(svr_pred)
confusion_matrix(tsr_12_output, svr_pred)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   1, 292,  42,   0,   1,   0,   0,   0],
       [  0,   0,   0, 794,  36,   1,   0,   0,   0],
       [  0,   0,   0,  27, 395,  24,   0,   0,   0],
       [  0,   0,   0,   0,  28, 356,  10,   0,   0],
       [  0,   0,   0,   0,   1,  41, 454,  14,   0],
       [  0,   0,   0,   0,   0,   0,  30, 248,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [8]:
svr_pred[svr_pred == -1] = 0
svr_pred[svr_pred == -2] = 0
svr_pred[svr_pred == 6] = 5
confusion_matrix(tsr_12_output, svr_pred)

array([[294,  42,   0,   1,   0,   0],
       [  0, 794,  36,   1,   0,   0],
       [  0,  27, 395,  24,   0,   0],
       [  0,   0,  28, 356,  10,   0],
       [  0,   0,   1,  41, 454,  14],
       [  0,   0,   0,   0,  30, 249]], dtype=int64)

In [9]:
svr1 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores1 = cross_val_score(svr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores1)
print("Mean of R^2:", svr_scores1.mean())
print("Std of R^2:", svr_scores1.std())

[0.71448973 0.75274534 0.82736483 0.81437323 0.77947162 0.82689984
 0.79493208 0.81629587 0.72042847 0.69645312]
Mean of R^2: 0.7743454126020682
Std of R^2: 0.04733782272086469


In [10]:
svr1.fit(tsr_12_input_nomrs,tsr_12_output)
svr_predict1 =svr1.predict(tsr_12_input_nomrs)
print(svr_predict1)
svr_predict1 = np.round(svr_predict1)
print(svr_predict1)
print("Accuracy of RF:", (tsr_12_output == svr_predict1).sum() / len(svr_predict1))

[1.11183691 0.87118922 0.94313173 ... 1.13251514 4.78957798 1.54101559]
[1. 1. 1. ... 1. 5. 2.]
Accuracy of RF: 0.5613156953879156


In [11]:
svr_pred1 = cross_val_predict(svr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
svr_pred1 = np.round(svr_pred1)
confusion_matrix(tsr_12_output, svr_pred1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   1,  55, 245,  30,   4,   0,   1,   0],
       [  0,   0,  14, 654, 133,  24,   6,   0,   0],
       [  0,   0,   1, 169, 193,  76,   7,   0,   0],
       [  0,   0,   0,  15,  99, 240,  35,   5,   0],
       [  0,   0,   0,   5,  18, 156, 277,  54,   0],
       [  0,   0,   0,   1,   6,  14,  79, 174,   5],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [12]:
svr_pred1[svr_pred1 == -5] = 0
svr_pred1[svr_pred1 == -4] = 0
confusion_matrix(tsr_12_output, svr_pred1)

array([[ 57, 245,  30,   4,   0,   1,   0],
       [ 14, 654, 133,  24,   6,   0,   0],
       [  1, 169, 193,  76,   7,   0,   0],
       [  0,  15,  99, 240,  35,   5,   0],
       [  0,   5,  18, 156, 277,  54,   0],
       [  0,   1,   6,  14,  79, 174,   5],
       [  0,   0,   0,   0,   0,   0,   0]], dtype=int64)

## RF

In [13]:
rfr = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores = cross_val_score(rfr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores)
print("Mean of R^2:", rfr_scores.mean())
print("Std of R^2:", rfr_scores.std())

[0.99995718 0.99966588 0.9999666  0.99991394 0.99951801 0.9999538
 0.99990526 0.99996546 0.99998622 0.99988793]
Mean of R^2: 0.999872028686078
Std of R^2: 0.0001468289738189413


In [14]:
rfr.fit(tsr_12_input,tsr_12_output)
rfr_predict =rfr.predict(tsr_12_input)
print(rfr_predict)
rfr_predict = np.round(rfr_predict)
print(rfr_predict)
print("Accuracy of RF:", (tsr_12_output == rfr_predict).sum() / len(rfr_predict))

[1. 1. 0. ... 0. 5. 1.]
[1. 1. 0. ... 0. 5. 1.]
Accuracy of RF: 1.0


In [15]:
rfr_pred = cross_val_predict(rfr,tsr_12_input,tsr_12_output,cv = 10)
rfr_pred = np.round(rfr_pred)
confusion_matrix(tsr_12_output, rfr_pred)

array([[337,   0,   0,   0,   0,   0],
       [  0, 831,   0,   0,   0,   0],
       [  0,   0, 446,   0,   0,   0],
       [  0,   0,   0, 394,   0,   0],
       [  0,   0,   0,   0, 510,   0],
       [  0,   0,   0,   0,   0, 279]], dtype=int64)

In [16]:
rfr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores1 = cross_val_score(rfr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores1)
print("Mean of R^2:", rfr_scores1.mean())
print("Std of R^2:", rfr_scores1.std())

[0.7847378  0.81460552 0.86070665 0.81994466 0.78999239 0.83301935
 0.83633177 0.86456348 0.77757829 0.78390643]
Mean of R^2: 0.8165386343479961
Std of R^2: 0.030388623691578846


In [17]:
rfr1.fit(tsr_12_input_nomrs,tsr_12_output)
rfr_predict1 =rfr1.predict(tsr_12_input_nomrs)
print(rfr_predict1)
rfr_predict1 = np.round(rfr_predict1)
print(rfr_predict1)
print("Accuracy of RF:", (tsr_12_output == rfr_predict1).sum() / len(rfr_predict1))

[0.93333333 0.86666667 0.         ... 0.4        4.86666667 1.2       ]
[1. 1. 0. ... 0. 5. 1.]
Accuracy of RF: 0.9349302824454773


In [18]:
rfr_pred1 = cross_val_predict(rfr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
rfr_pred1 = np.round(rfr_pred1)
confusion_matrix(tsr_12_output, rfr_pred1)

array([[209, 107,  14,   6,   0,   1],
       [ 44, 593, 156,  34,   4,   0],
       [  0, 104, 265,  68,   9,   0],
       [  0,  12,  93, 236,  52,   1],
       [  0,   6,  17,  95, 364,  28],
       [  0,   1,   2,  12,  86, 178]], dtype=int64)

## XGBoost

In [19]:
xgbr = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores = cross_val_score(xgbr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores)
print("Mean of R^2:", xgbr_scores.mean())
print("Std of R^2:", xgbr_scores.std())

[0.99993885 0.99995166 0.99995083 0.99995068 0.99994955 0.99994612
 0.99995135 0.99994278 0.99994875 0.99993334]
Mean of R^2: 0.9999463923219803
Std of R^2: 5.884427122176203e-06


In [20]:
xgbr.fit(tsr_12_input,tsr_12_output)
xgbr_predict =xgbr.predict(tsr_12_input)
print(xgbr_predict)
xgbr_predict = np.round(xgbr_predict)
print(xgbr_predict)
print("Accuracy of RF:", (tsr_12_output == xgbr_predict).sum() / len(xgbr_predict))

[9.9760783e-01 9.9760783e-01 2.4193309e-03 ... 2.4193309e-03 4.9781404e+00
 9.9760783e-01]
[1. 1. 0. ... 0. 5. 1.]
Accuracy of RF: 1.0


In [21]:
xgbr_pred = cross_val_predict(xgbr,tsr_12_input,tsr_12_output,cv = 10)
xgbr_pred = np.round(xgbr_pred)
confusion_matrix(tsr_12_output, xgbr_pred)

array([[337,   0,   0,   0,   0,   0],
       [  0, 831,   0,   0,   0,   0],
       [  0,   0, 446,   0,   0,   0],
       [  0,   0,   0, 394,   0,   0],
       [  0,   0,   0,   0, 510,   0],
       [  0,   0,   0,   0,   0, 279]], dtype=int64)

In [22]:
xgbr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
xgbr_scores1 = cross_val_score(xgbr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores1)
print("Mean of R^2:", xgbr_scores1.mean())
print("Std of R^2:", xgbr_scores1.std())

[0.7847378  0.81460552 0.86070665 0.81994466 0.78999239 0.83301935
 0.83633177 0.86456348 0.77757829 0.78390643]
Mean of R^2: 0.8165386343479961
Std of R^2: 0.030388623691578846


In [23]:
xgbr1.fit(tsr_12_input_nomrs,tsr_12_output)
xgbr_predict1 =xgbr1.predict(tsr_12_input_nomrs)
print(xgbr_predict1)
xgbr_predict1 = np.round(xgbr_predict1)
print(xgbr_predict1)
print("Accuracy of RF:", (tsr_12_output == xgbr_predict1).sum() / len(xgbr_predict1))

[0.93333333 0.86666667 0.         ... 0.4        4.86666667 1.2       ]
[1. 1. 0. ... 0. 5. 1.]
Accuracy of RF: 0.9349302824454773


In [24]:
xgbr_pred1 = cross_val_predict(xgbr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
xgbr_pred1 = np.round(xgbr_pred1)
confusion_matrix(tsr_12_output, xgbr_pred1)

array([[209, 107,  14,   6,   0,   1],
       [ 44, 593, 156,  34,   4,   0],
       [  0, 104, 265,  68,   9,   0],
       [  0,  12,  93, 236,  52,   1],
       [  0,   6,  17,  95, 364,  28],
       [  0,   1,   2,  12,  86, 178]], dtype=int64)

# 2 classes

In [25]:
tsr_12_output[(tsr_12_output == 0)|(tsr_12_output == 1)|(tsr_12_output == 2)] = 0
tsr_12_output[(tsr_12_output == 3)|(tsr_12_output == 4)|(tsr_12_output == 5)] = 1

## SVM

In [26]:
svr2 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores2 = cross_val_score(svr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores2)
print("Mean of R^2:", svr_scores2.mean())
print("Std of R^2:", svr_scores2.std())

[0.74688802 0.80922849 0.82543556 0.78713999 0.83460751 0.83668233
 0.82601752 0.78014965 0.76469575 0.75393518]
Mean of R^2: 0.7964779995334053
Std of R^2: 0.0325221903342801


In [27]:
svr2.fit(tsr_12_input,tsr_12_output)
svr_predict2 =svr2.predict(tsr_12_input)
print(svr_predict2)
svr_predict2 = np.round(svr_predict2)
print(svr_predict2)
print("Accuracy of RF:", (tsr_12_output == svr_predict2).sum() / len(svr_predict2))

[-1.27222339e-04 -4.25323644e-02 -1.77274041e-01 ... -1.54786124e-01
  1.20158204e+00  2.20727857e-02]
[-0. -0. -0. ... -0.  1.  0.]
Accuracy of RF: 0.9678226671433678


In [28]:
svr_pred2 = cross_val_predict(svr2,tsr_12_input,tsr_12_output,cv = 10)
svr_pred2 = np.round(svr_pred2)
confusion_matrix(tsr_12_output, svr_pred2)

array([[   0,    0,    0],
       [   3, 1576,   35],
       [   0,   42, 1141]], dtype=int64)

In [29]:
svr_pred2[svr_pred2 == -1] = 0
confusion_matrix(tsr_12_output, svr_pred2)

array([[1579,   35],
       [  42, 1141]], dtype=int64)

In [30]:
svr3 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores3 = cross_val_score(svr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores3)
print("Mean of R^2:", svr_scores3.mean())
print("Std of R^2:", svr_scores3.std())

[0.60997352 0.64152925 0.73357294 0.68732289 0.70386453 0.77256948
 0.69745339 0.69620375 0.64261836 0.58812289]
Mean of R^2: 0.6773230997995845
Std of R^2: 0.053647301204474515


In [31]:
svr3.fit(tsr_12_input_nomrs,tsr_12_output)
svr_predict3 =svr3.predict(tsr_12_input_nomrs)
print(svr_predict3)
svr_predict3 = np.round(svr_predict3)
print(svr_predict3)
print("Accuracy of RF:", (tsr_12_output == svr_predict3).sum() / len(svr_predict3))

[ 0.02395908 -0.01231632  0.01042118 ...  0.072532    1.18530539
  0.08558349]
[ 0. -0.  0. ...  0.  1.  0.]
Accuracy of RF: 0.9095459420808009


In [32]:
svr_pred3 = cross_val_predict(svr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
svr_pred3 = np.round(svr_pred3)
confusion_matrix(tsr_12_output, svr_pred3)

array([[   0,    0,    0],
       [   1, 1474,  139],
       [   0,  127, 1056]], dtype=int64)

In [33]:
svr_pred3[svr_pred3 == -1] = 0
confusion_matrix(tsr_12_output, svr_pred3)

array([[1475,  139],
       [ 127, 1056]], dtype=int64)

## RF

In [34]:
rfr2 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores2 = cross_val_score(rfr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores2)
print("Mean of R^2:", rfr_scores2.mean())
print("Std of R^2:", rfr_scores2.std())

[1.         0.99980424 1.         0.99992919 1.         1.
 1.         1.         1.         1.        ]
Mean of R^2: 0.9999733432645282
Std of R^2: 6.019117035794216e-05


In [35]:
rfr2.fit(tsr_12_input,tsr_12_output)
rfr_predict2 =rfr2.predict(tsr_12_input)
print(rfr_predict2)
rfr_predict2 = np.round(rfr_predict2)
print(rfr_predict2)
print("Accuracy of RF:", (tsr_12_output == rfr_predict2).sum() / len(rfr_predict2))

[0. 0. 0. ... 0. 1. 0.]
[0. 0. 0. ... 0. 1. 0.]
Accuracy of RF: 1.0


In [36]:
rf_pred2 = cross_val_predict(rfr2,tsr_12_input,tsr_12_output,cv = 10)
rf_pred2 = np.round(rf_pred2)
confusion_matrix(tsr_12_output, rf_pred2)

array([[1614,    0],
       [   0, 1183]], dtype=int64)

In [37]:
rfr3 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores3 = cross_val_score(rfr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores3)
print("Mean of R^2:", rfr_scores3.mean())
print("Std of R^2:", rfr_scores3.std())

[0.67436952 0.70433759 0.7949208  0.74424277 0.75273339 0.79387526
 0.7098545  0.73176183 0.6800626  0.60852235]
Mean of R^2: 0.7194680609420423
Std of R^2: 0.054146120458681925


In [38]:
rfr3.fit(tsr_12_input_nomrs,tsr_12_output)
rfr_predict3 =rfr3.predict(tsr_12_input_nomrs)
print(rfr_predict3)
rfr_predict3 = np.round(rfr_predict3)
print(rfr_predict3)
print("Accuracy of RF:", (tsr_12_output == rfr_predict3).sum() / len(rfr_predict3))

[0.         0.         0.         ... 0.         1.         0.06666667]
[0. 0. 0. ... 0. 1. 0.]
Accuracy of RF: 0.997139792634966


In [39]:
rfr_pred3 = cross_val_predict(rfr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
rfr_pred3 = np.round(rfr_pred3)
confusion_matrix(tsr_12_output, rfr_pred3)

array([[1462,  152],
       [  93, 1090]], dtype=int64)

## XGBoost

In [40]:
xgbr2 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores2 = cross_val_score(xgbr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores2)
print("Mean of R^2:", xgbr_scores2.mean())
print("Std of R^2:", xgbr_scores2.std())

[0.99997706 0.9999766  0.99997648 0.99997461 0.99997654 0.999977
 0.99997628 0.99997644 0.99997609 0.99997706]
Mean of R^2: 0.9999764165186296
Std of R^2: 6.781968422816103e-07


In [41]:
xgbr2.fit(tsr_12_input,tsr_12_output)
xgbr_predict2 =xgbr2.predict(tsr_12_input)
print(xgbr_predict2)
xgbr_predict2 = np.round(xgbr_predict2)
print(xgbr_predict2)
print("Accuracy of RF:", (tsr_12_output == xgbr_predict2).sum() / len(xgbr_predict2))

[0.00238325 0.00238325 0.00238325 ... 0.00238325 0.9976133  0.00238325]
[0. 0. 0. ... 0. 1. 0.]
Accuracy of RF: 1.0


In [42]:
xgbr_pred2 = cross_val_predict(xgbr2,tsr_12_input,tsr_12_output,cv = 10)
xgbr_pred2 = np.round(xgbr_pred2)
confusion_matrix(tsr_12_output, xgbr_pred2)

array([[1614,    0],
       [   0, 1183]], dtype=int64)

In [43]:
xgbr3 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores3 = cross_val_score(xgbr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores3)
print("Mean of R^2:", xgbr_scores3.mean())
print("Std of R^2:", xgbr_scores3.std())

[0.69106629 0.72264037 0.82978793 0.75276784 0.76755001 0.79607017
 0.68396137 0.74830537 0.67251316 0.6428272 ]
Mean of R^2: 0.7307489717278237
Std of R^2: 0.055833807794982114


In [44]:
xgbr3.fit(tsr_12_input_nomrs,tsr_12_output)
xgbr_predict3 =xgbr3.predict(tsr_12_input_nomrs)
print(xgbr_predict3)
xgbr_predict3 = np.round(xgbr_predict3)
print(xgbr_predict3)
print("Accuracy of RF:", (tsr_12_output == xgbr_predict3).sum() / len(xgbr_predict3))

[0.00315062 0.0029903  0.00510203 ... 0.03138744 0.98297805 0.01206112]
[0. 0. 0. ... 0. 1. 0.]
Accuracy of RF: 0.9910618519842689


In [45]:
xgbr_pred3 = cross_val_predict(xgbr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
xgbr_pred3 = np.round(xgbr_pred3)
confusion_matrix(tsr_12_output, xgbr_pred3)

array([[1472,  142],
       [  95, 1088]], dtype=int64)

# Summary

## Mean & Std

In [46]:
svr_mean = np.array([svr_scores.mean(), svr_scores.std(), svr_scores[0], svr_scores[1], svr_scores[2], svr_scores[3],
                     svr_scores[4], svr_scores[5], svr_scores[6], svr_scores[7], svr_scores[8], svr_scores[9]])
rfr_mean = np.array([rfr_scores.mean(), rfr_scores.std(), rfr_scores[0], rfr_scores[1], rfr_scores[2], rfr_scores[3],rfr_scores[4], 
                    rfr_scores[5], rfr_scores[6], rfr_scores[7], rfr_scores[8], rfr_scores[9]])
xgbr_mean = np.array([xgbr_scores.mean(), xgbr_scores.std(), xgbr_scores[0], xgbr_scores[1], xgbr_scores[2], xgbr_scores[3],
                     xgbr_scores[4], xgbr_scores[5], xgbr_scores[6], xgbr_scores[7], xgbr_scores[8], xgbr_scores[9]])
svr_mean2 = np.array([svr_scores2.mean(), svr_scores2.std(), svr_scores2[0], svr_scores2[1], svr_scores2[2], svr_scores2[3],
                     svr_scores2[4], svr_scores2[5], svr_scores2[6], svr_scores2[7], svr_scores2[8], svr_scores2[9]])
rfr_mean2 = np.array([rfr_scores2.mean(), rfr_scores2.std(), rfr_scores2[0], rfr_scores2[1], rfr_scores2[2], rfr_scores2[3],rfr_scores2[4], 
                    rfr_scores2[5], rfr_scores2[6], rfr_scores2[7], rfr_scores2[8], rfr_scores2[9]])
xgbr_mean2 = np.array([xgbr_scores2.mean(), xgbr_scores2.std(), xgbr_scores2[0], xgbr_scores2[1], xgbr_scores2[2], xgbr_scores2[3],
                     xgbr_scores2[4], xgbr_scores2[5], xgbr_scores2[6], xgbr_scores2[7], xgbr_scores2[8], xgbr_scores2[9]])

In [47]:
tsr_12_mean = pd.DataFrame([svr_mean, rfr_mean, xgbr_mean, svr_mean2, rfr_mean2, xgbr_mean2]).T
tsr_12_mean.index = ["Mean", "Std", "R^2_1", "R^2_2", "R^2_3", "R^2_4", "R^2_5", "R^2_6", "R^2_7", "R^2_8", "R^2_9", "R^2_10"]
tsr_12_mean.columns = ["svr", 'rfr', 'xgbr', 'svr2', 'rfr2', 'xgbr2']

In [48]:
csv_save = os.path.join(".", "tsr_12_mean_regression.csv")
tsr_12_mean.to_csv(csv_save, index = True)